In [1]:
import netron
from IPython.display import IFrame

def showInNetron(model_filename):
    netron.start(model_filename, address=("0.0.0.0",8081))
    return IFrame(src="http://0.0.0.0:8081/",width="100%",height=400)

tidy-up

In [2]:
import onnx
from finn.util.test import get_test_model_trained
import brevitas.onnx as bo
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.infer_shapes import InferShapes
from finn.transformation.fold_constants import FoldConstants
from finn.transformation.general import GiveReadableTensorNames,GiveRandomTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs

model = ModelWrapper("./save/ECG_test.onnx")
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(RemoveStaticGraphInputs())
model.save("./test/ECG_AF_2D_w2a2_tidy.onnx")

pre-process

In [4]:
from finn.util.pytorch import ToTensor
from finn.transformation.merge_onnx_models import MergeONNXModels
from finn.core.datatype import DataType

model = ModelWrapper("./test/ECG_AF_2D_w2a2_tidy.onnx")
global_inp_name = model.graph.input[0].name
ishape = model.get_tensor_shape(global_inp_name)
# preprocessing: torchvision's ToTensor divides uint8 inputs by 255
totensor_pyt = ToTensor()
chkpt_preproc_name = "./test/ECG_AF_2D_w2a2_preproc.onnx"
bo.export_finn_onnx(totensor_pyt, ishape, chkpt_preproc_name)

# join preprocessing and core model
pre_model = ModelWrapper(chkpt_preproc_name)
model = model.transform(MergeONNXModels(pre_model))
# add input quantization annotation: UINT8 for all BNN-PYNQ models
global_inp_name = model.graph.input[0].name
model.set_tensor_datatype(global_inp_name, DataType.UINT8)

/workspace/finn-base/src/finn/transformation/infer_data_layouts.py:114: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")


post-process

In [5]:
from finn.transformation.insert_topk import InsertTopK
from finn.transformation.infer_datatypes import InferDataTypes

# postprocessing: insert Top-1 node at the end
model = model.transform(InsertTopK(k=1))
chkpt_name = "./test/ECG_AF_2D_w2a2_post.onnx"
# tidy-up again
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
#model = model.transform(GiveRandomTensorNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())
model.save(chkpt_name)

In [6]:
showInNetron("./test/ECG_AF_2D_w2a2_post.onnx")

Serving './test/ECG_AF_2D_w2a2_post.onnx' at http://0.0.0.0:8081


streamline

In [7]:
from finn.transformation.streamline import Streamline
from finn.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from finn.transformation.bipolar_to_xnor import ConvertBipolarMatMulToXnorPopcount
import finn.transformation.streamline.absorb as absorb
from finn.transformation.streamline.reorder import MakeMaxPoolNHWC, MoveScalarLinearPastInvariants, MoveFlattenPastAffine
from finn.transformation.infer_data_layouts import InferDataLayouts
from finn.transformation.general import RemoveUnusedTensors

model = ModelWrapper("./test/ECG_AF_2D_w2a2_post.onnx")
model = model.transform(MoveScalarLinearPastInvariants())
model = model.transform(Streamline())
model = model.transform(LowerConvsToMatMul())
#model = model.transform(MoveFlattenPastAffine())
model = model.transform(MakeMaxPoolNHWC())

#model.save("./test/ECG_AF_2D_w2a2_test1.onnx")

model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())

model = model.transform(MakeMaxPoolNHWC())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
#model.save("./test/ECG_AF_1D_w2a2_test2.onnx")

model = model.transform(ConvertBipolarMatMulToXnorPopcount())
model = model.transform(Streamline())
# absorb final add-mul nodes into TopK
model = model.transform(absorb.AbsorbScalarMulAddIntoTopK())
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())



model.save("./test/ECG_AF_2D_w2a2_streamlined.onnx")

convert to HLS layer and dataflow partitioning

In [8]:
import os
import finn.transformation.fpgadataflow.convert_to_hls_layers as to_hls
from finn.transformation.fpgadataflow.create_dataflow_partition import (
    CreateDataflowPartition,
)
from finn.transformation.move_reshape import RemoveCNVtoFCFlatten
from finn.custom_op.registry import getCustomOp
from finn.transformation.infer_data_layouts import InferDataLayouts

output_dir = "./test/output_ipstitch/intermediate_models"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)


# choose the memory mode for the MVTU units, decoupled or const
mem_mode = "const"

model = ModelWrapper("./test/ECG_AF_2D_w2a2_streamlined.onnx")



#'''
model = model.transform(to_hls.InferQuantizedStreamingFCLayer(mem_mode))
# TopK to LabelSelect
model = model.transform(to_hls.InferLabelSelectLayer())
# input quantization (if any) to standalone thresholding

model = model.transform(to_hls.InferVVAU())
model = model.transform(to_hls.InferStreamingMaxPool())
model = model.transform(to_hls.InferPool_Batch())

model = model.transform(to_hls.InferThresholdingLayer())
model = model.transform(to_hls.InferConvInpGen())
model = model.transform(to_hls.InferStreamingMaxPool())
# get rid of Reshape(-1, 1) operation between hlslib nodes
model = model.transform(RemoveCNVtoFCFlatten())
# get rid of Tranpose -> Tranpose identity seq
model = model.transform(absorb.AbsorbConsecutiveTransposes())
# infer tensor data layouts
model = model.transform(InferDataLayouts())


#'''
model = model.transform(to_hls.InferPool_Batch())
model = model.transform(to_hls.InferConvInpGen())
model = model.transform(to_hls.InferVVAU())
model = model.transform(to_hls.InferQuantizedStreamingFCLayer(mem_mode))
model = model.transform(to_hls.InferChannelwiseLinearLayer())
model = model.transform(to_hls.InferLabelSelectLayer())
model = model.transform(InferShapes())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())

#'''

model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())
model = model.transform(RemoveUnusedTensors())




model.save("./test/ECG_AF_2D_w2a2_hls.onnx")
parent_model = model.transform(CreateDataflowPartition())
parent_model.save("./test/output_ipstitch/intermediate_models/dataflow_parent.onnx")
sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
sdp_node = getCustomOp(sdp_node)
dataflow_model_filename = sdp_node.get_nodeattr("model")
# save the dataflow partition with a different name for easier access
dataflow_model = ModelWrapper(dataflow_model_filename)
dataflow_model.save("./test/output_ipstitch/intermediate_models/dataflow_model.onnx")

/workspace/finn/src/finn/custom_op/fpgadataflow/streamingfclayer_batch.py:669: UserWarning: Setting 0-valued first threshold to 1 to avoid vivado_hls bug
  warnings.warn(
/workspace/finn/src/finn/custom_op/fpgadataflow/vector_vector_activate_batch.py:288: UserWarning: Setting 0-valued first threshold to 1 to avoid vivado_hls bug
  warnings.warn(


In [9]:
showInNetron("./test/ECG_AF_2D_w2a2_hls.onnx")

Stopping http://0.0.0.0:8081
Serving './test/ECG_AF_2D_w2a2_hls.onnx' at http://0.0.0.0:8081


In [10]:
showInNetron("./test/output_ipstitch/intermediate_models/dataflow_parent.onnx")

Stopping http://0.0.0.0:8081
Serving './test/output_ipstitch/intermediate_models/dataflow_parent.onnx' at http://0.0.0.0:8081


In [11]:
showInNetron("./test/output_ipstitch/intermediate_models/dataflow_model.onnx")

Stopping http://0.0.0.0:8081
Serving './test/output_ipstitch/intermediate_models/dataflow_model.onnx' at http://0.0.0.0:8081


start compiling

In [12]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg

model_file = "./test/ECG_AF_2D_w2a2_hls.onnx"

rtlsim_output_dir = "./test/output_ipstitch"

cfg = build.DataflowBuildConfig(
    output_dir          = rtlsim_output_dir,
    #target_fps          = 10000,
    synth_clk_period_ns = 20.0, #50 MHz
    fpga_part           = "xc7a15tfgg484-1",
    #fpga_part           = "xc7a50tfgg484-1",,
    
    default_mem_mode = build_cfg.ComputeEngineMemMode.CONST,
    large_fifo_mem_style = build_cfg.LargeFIFOMemStyle.BRAM,
    
    #folding_config_file = "./test/simple_test.json",
    folding_config_file = "./test/test_multi.json",
    #folding_config_file = "./test/final_hw_config.json",

    steps = ["step_create_dataflow_partition",
             "step_target_fps_parallelization",
             "step_apply_folding_config",
             "step_generate_estimate_reports",
             "step_hls_codegen",
             "step_hls_ipgen",
             "step_set_fifo_depths",
            "step_create_stitched_ip",],
             
   

    
    generate_outputs=[
        build_cfg.DataflowOutputType.STITCHED_IP,
    ]
)

build.build_dataflow_cfg(model_file, cfg)

Building dataflow accelerator from ./test/ECG_AF_2D_w2a2_hls.onnx
Intermediate outputs will be generated in /tmp/finn_dev_wenlong
Final outputs will be generated in ./test/output_ipstitch
Build log is at ./test/output_ipstitch/build_dataflow.log
Running step: step_create_dataflow_partition [1/8]
Running step: step_target_fps_parallelization [2/8]
Running step: step_apply_folding_config [3/8]
Running step: step_generate_estimate_reports [4/8]
Running step: step_hls_codegen [5/8]
Running step: step_hls_ipgen [6/8]
Running step: step_set_fifo_depths [7/8]
Running step: step_create_stitched_ip [8/8]
Completed successfully


0